# Import Modules

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader

import utils

# Prepare Data

In [2]:
source_X = pd.read_csv("./deep_occupancy_detection/data/1_X_train.csv").values
target_X = pd.read_csv("./deep_occupancy_detection/data/2_X_train.csv").values

source_y_task = pd.read_csv("./deep_occupancy_detection/data/1_Y_train.csv").values.reshape(-1)
target_y_task = pd.read_csv("./deep_occupancy_detection/data/2_Y_train.csv").values.reshape(-1)

scaler = preprocessing.StandardScaler()
scaler.fit(source_X)
source_X = scaler.transform(source_X)
target_X = scaler.transform(target_X)

source_target_X = np.concatenate([source_X, target_X], axis=0)
source_target_y_domain = np.concatenate([np.zeros(source_X.shape[0]), np.ones(target_X.shape[0])], axis=0)

In [3]:
# for 2. Marginal Distribution Discrepancy between Source and Target
source_target_X = torch.Tensor(source_target_X)
source_target_y_domain = torch.Tensor(source_target_y_domain)

source_target_X = source_target_X.to(utils.DEVICE)
source_target_y_domain = source_target_y_domain.to(utils.DEVICE)

source_target_ds = TensorDataset(source_target_X, source_target_y_domain)
source_target_loader = DataLoader(source_target_ds, batch_size=16, shuffle=True)

In [4]:
# for 3. Common Model Minimizin Loss of Both Domains
source_X = torch.Tensor(source_X)
target_X = torch.Tensor(target_X)
source_y_task = torch.Tensor(source_y_task)
target_y_task = torch.Tensor(target_y_task)

source_X = source_X.to(utils.DEVICE)
target_X = target_X.to(utils.DEVICE)
source_y_task = source_y_task.to(utils.DEVICE)
target_y_task = target_y_task.to(utils.DEVICE)

source_ds = TensorDataset(source_X, source_y_task)
target_ds = TensorDataset(target_X, target_y_task)

source_loader = DataLoader(source_ds, batch_size=16, shuffle=True)
target_loader = DataLoader(target_ds, batch_size=16, shuffle=True)

# 1. 

# 2. Marginal Distribution Discrepancy between Source and Target

In [5]:
criterion = nn.BCELoss()
num_repeats = 10
num_epochs = 100

In [15]:
accs = []
for _ in range(num_repeats):
    domain_classifier = utils.Decoder(input_size=source_target_X.shape[1], output_size=1).to(utils.DEVICE)
    optimizer = optim.Adam(domain_classifier.parameters(), lr=0.001)

    for _ in range(num_epochs):
        for source_target_X_batch, source_target_y_domain_batch in source_target_loader:
            # Forward
            pred_y = domain_classifier(source_target_X_batch)
            pred_y = torch.sigmoid(pred_y).reshape(-1)
            loss = criterion(pred_y, source_target_y_domain_batch)

            # Backward
            optimizer.zero_grad()
            loss.backward()

            # Update Params
            optimizer.step()

    # Evaluation
    pred_y = domain_classifier(source_target_X)
    pred_y = torch.sigmoid(pred_y).reshape(-1)
    pred_y = pred_y > 0.5

    acc = sum(pred_y == source_target_y_domain) / source_target_y_domain.shape[0]
    accs.append(acc.item())

print(f"Domain Classification Accuracy: {np.mean(accs)}")

Domain Classification Accuracy: 0.7404029846191407


# 3. Common Model Minimizin Loss of Both Domains

In [10]:
losses = []
for _ in range(num_repeats):
    task_classifier = utils.Decoder(input_size=source_X.shape[1], output_size=1).to(utils.DEVICE)
    optimizer = optim.Adam(task_classifier.parameters(), lr=0.001)
    for _ in range(num_epochs):
        for (source_X_batch, source_y_task_batch), (target_X_batch, target_y_task_batch) in zip(source_loader, target_loader):
            # Forward
            pred_source_y_task = task_classifier(source_X_batch)
            pred_target_y_task = task_classifier(target_X_batch)
            pred_source_y_task = torch.sigmoid(pred_source_y_task).reshape(-1)
            pred_target_y_task = torch.sigmoid(pred_target_y_task).reshape(-1)
            loss = criterion(pred_source_y_task, source_y_task_batch)
            loss += criterion(pred_target_y_task, target_y_task_batch)

            # Backward
            optimizer.zero_grad()
            loss.backward()

            # Update Params
            optimizer.step()

    # Evaluation
    pred_y = task_classifier(source_X)
    pred_y = torch.sigmoid(pred_y).reshape(-1)
    loss = criterion(pred_y, source_y_task)

    pred_y = task_classifier(target_X)
    pred_y = torch.sigmoid(pred_y).reshape(-1)
    loss += criterion(pred_y, target_y_task)
    losses.append(loss.item())

print(f"Common Model's Cross Entropy Loss: {np.mean(losses)}")

Common Model's Cross Entropy Loss: 0.6571887731552124
